In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"./Aleppo2017_processed.csv",encoding='utf-8')
df.head()

,id,time,gl
0,183,2015-05-16 05:35:41,162.0
1,183,2015-05-16 05:30:41,164.0
2,183,2015-05-16 05:25:41,168.0
3,183,2015-05-16 05:20:41,169.0
4,183,2015-05-16 05:15:41,170.0


In [3]:
ID = df['id'].value_counts().index
print(ID)

Index([263,  77, 193, 277, 229, 245, 155, 251, 111, 164,
       ...
       128, 264, 186, 162, 249,  39,  52, 266, 223, 289],
      dtype='int64', name='id', length=226)


In [18]:
for i in ID[:10]:
    #获取每个被试的数据
    data = df[df['id'] == i].copy()
    
    #对每个被试的数据根据时间进行排序
    #1、先将时间列设成datetime类型
    data['time'] = pd.to_datetime(data['time']) 
    
    #2、根据时间排序
    data = data.sort_values(by='time')
    
    #3、统计每个被试每天中有多少血糖数据
        #3.1 提取时间的年月日信息, 一天中的血糖数据的年月日信息相同
    data['date'] = data['time'].dt.date
        #3.2 统计相同的年月日中有多少血糖数据
    daily_data_count = data.groupby('date')['gl'].count()
    
    #4、筛选数据
        #4.1 连续7天
        #4.2 每天的数据量大于260
    continuous_days = 0    #记录连续的天数
    target_dates = []    #存放筛选出的数据(time)
    
    #此段for循环找出参与者第一个连续7天，且每天血糖数据大于260的日期
    for date,count in daily_data_count.items():
        if continuous_days >= 7:
            break
        else:
            if count > 260:
                target_dates.append(date)
                continuous_days += 1
            else:
                continuous_days = 0
                target_dates = []
                

    #5、处理target_dates中记录的日期对应的数据
    #5.1 获取处理target_dates中每天的数据
    for j in target_dates:
        data_of_day = data[data['date'] == j]
        
        #5.2 如果这天中的数据满足288条则直接写入csv,如果不满足288则需要进行插值处理
        if data_of_day.shape[0] == 288:
            print("{}这天测够了288条数据".format(j))
            data_of_day = data_of_day.set_index('time')
            data_of_day[['id','gl']].to_csv(r"C:\Users\admin\Desktop\week_of_data2.csv", mode='a', header=False,index=True)
        else:
            print("{}这天的血糖记录数不满足288".format(j))
            print("day_data.shape",data_of_day.shape)
            #设置秒为0
            data_of_day.loc[:,'time'] = data_of_day['time'].apply(lambda dt: dt.replace(second=0))
            data_day = []
            for k in range(24):
                hour_data = data_of_day[data_of_day['time'].dt.hour == k].copy()    #某些小时可能没有数据
                
                if hour_data.empty:
                    print("数据在 {}-{}  小时时间范围内为空".format(k,k+1))
                    
                    start_date = datetime.datetime.combine(j, datetime.time(k, 0))
                    
                    full_time_index = pd.date_range(start=start_date, periods=12, freq='5T')
                    
                    data_empty = {'gl': [0] * len(full_time_index),'id':[i] * len(full_time_index)}
                    
                    data_day_padding = pd.DataFrame(data_empty, index=full_time_index)
                    
                    data_day.append(data_day_padding)
                    
                else:
                    minute = hour_data.iloc[0]['time'].minute % 5

                    # 设置一小时内的起始时间
                    start_date = hour_data.iloc[0]['time'].replace(hour=k, minute=minute)
    #                 print(start_date)

                    # 使用完整的时间序列重新索引
                    full_time_index = pd.date_range(start=start_date, periods=12, freq='5T')

                    #删除重复时间的记录
                    data_unique_times = hour_data[~hour_data['time'].duplicated(keep='first')]

                    # 将时间戳设置为索引
                    data_unique_times = data_unique_times.set_index('time')

                    # 使用完整的时间序列重新索引
                    hour_data_interpolated = data_unique_times.reindex(index=full_time_index)

                    # 进行线性插值interpolate()
                    hour_data_interpolated['gl'] = hour_data_interpolated['gl'].interpolate()
                    hour_data_interpolated['id'] = i
                    hour_data_interpolated = hour_data_interpolated.fillna({'gl': 0})
                    print("线性插值后的数据:", hour_data_interpolated)
                    data_day.append(hour_data_interpolated)
                
            data_day_all = pd.concat(data_day)
            data_day_all.to_csv(r"C:\Users\admin\Desktop\week_of_data2.csv", mode='a', header=False,index=True)
            
            data_day = []
    break

2015-05-20这天的血糖记录数不满足288
day_data.shape (270, 4)
线性插值后的数据:                       id     gl        date
2015-05-20 00:00:00  263  240.0  2015-05-20
2015-05-20 00:05:00  263  234.0  2015-05-20
2015-05-20 00:10:00  263  228.0  2015-05-20
2015-05-20 00:15:00  263  227.0  2015-05-20
2015-05-20 00:20:00  263  228.0  2015-05-20
2015-05-20 00:25:00  263  226.0  2015-05-20
2015-05-20 00:30:00  263  224.0  2015-05-20
2015-05-20 00:35:00  263  219.0  2015-05-20
2015-05-20 00:40:00  263  213.0  2015-05-20
2015-05-20 00:45:00  263  205.0  2015-05-20
2015-05-20 00:50:00  263  200.0  2015-05-20
2015-05-20 00:55:00  263  200.0  2015-05-20
线性插值后的数据:                       id     gl        date
2015-05-20 01:00:00  263  198.0  2015-05-20
2015-05-20 01:05:00  263  195.0  2015-05-20
2015-05-20 01:10:00  263  190.0  2015-05-20
2015-05-20 01:15:00  263  185.0  2015-05-20
2015-05-20 01:20:00  263  180.0  2015-05-20
2015-05-20 01:25:00  263  175.0  2015-05-20
2015-05-20 01:30:00  263  171.0  2015-05-20
2015-05

线性插值后的数据:                       id     gl        date
2015-05-22 23:00:00  263  118.0  2015-05-22
2015-05-22 23:05:00  263  116.0  2015-05-22
2015-05-22 23:10:00  263  113.0  2015-05-22
2015-05-22 23:15:00  263  109.0  2015-05-22
2015-05-22 23:20:00  263  100.0  2015-05-22
2015-05-22 23:25:00  263   92.0  2015-05-22
2015-05-22 23:30:00  263   87.0  2015-05-22
2015-05-22 23:35:00  263   84.0  2015-05-22
2015-05-22 23:40:00  263   82.0  2015-05-22
2015-05-22 23:45:00  263   79.0  2015-05-22
2015-05-22 23:50:00  263   77.0  2015-05-22
2015-05-22 23:55:00  263   82.0  2015-05-22
2015-05-23这天的血糖记录数不满足288
day_data.shape (367, 4)
线性插值后的数据:                       id     gl        date
2015-05-23 00:00:00  263   87.0  2015-05-23
2015-05-23 00:05:00  263   86.0  2015-05-23
2015-05-23 00:10:00  263   85.0  2015-05-23
2015-05-23 00:15:00  263   86.0  2015-05-23
2015-05-23 00:20:00  263   95.0  2015-05-23
2015-05-23 00:25:00  263  100.0  2015-05-23
2015-05-23 00:30:00  263  103.0  2015-05-23
2015-05

线性插值后的数据:                       id     gl        date
2015-05-24 20:03:00  263  293.0  2015-05-24
2015-05-24 20:08:00  263  285.0  2015-05-24
2015-05-24 20:13:00  263  283.0  2015-05-24
2015-05-24 20:18:00  263  284.0  2015-05-24
2015-05-24 20:23:00  263  286.0  2015-05-24
2015-05-24 20:28:00  263  287.0  2015-05-24
2015-05-24 20:33:00  263  288.0  2015-05-24
2015-05-24 20:38:00  263  288.0  2015-05-24
2015-05-24 20:43:00  263  289.0  2015-05-24
2015-05-24 20:48:00  263  291.0  2015-05-24
2015-05-24 20:53:00  263  291.0  2015-05-24
2015-05-24 20:58:00  263  292.0  2015-05-24
线性插值后的数据:                       id     gl        date
2015-05-24 21:03:00  263  292.0  2015-05-24
2015-05-24 21:08:00  263  291.0  2015-05-24
2015-05-24 21:13:00  263  289.0  2015-05-24
2015-05-24 21:18:00  263  285.0  2015-05-24
2015-05-24 21:23:00  263  278.0  2015-05-24
2015-05-24 21:28:00  263  270.0  2015-05-24
2015-05-24 21:33:00  263  260.0  2015-05-24
2015-05-24 21:38:00  263  251.0  2015-05-24
2015-05-24 2

线性插值后的数据:                       id     gl        date
2015-05-26 18:03:00  263  240.0  2015-05-26
2015-05-26 18:08:00  263  245.0  2015-05-26
2015-05-26 18:13:00  263  252.0  2015-05-26
2015-05-26 18:18:00  263  255.0  2015-05-26
2015-05-26 18:23:00  263  260.0  2015-05-26
2015-05-26 18:28:00  263  268.0  2015-05-26
2015-05-26 18:33:00  263  276.0  2015-05-26
2015-05-26 18:38:00  263  285.0  2015-05-26
2015-05-26 18:43:00  263  295.0  2015-05-26
2015-05-26 18:48:00  263  299.0  2015-05-26
2015-05-26 18:53:00  263  306.0  2015-05-26
2015-05-26 18:58:00  263  310.0  2015-05-26
线性插值后的数据:                       id     gl        date
2015-05-26 19:03:00  263  312.0  2015-05-26
2015-05-26 19:08:00  263  309.0  2015-05-26
2015-05-26 19:13:00  263  300.0  2015-05-26
2015-05-26 19:18:00  263  291.0  2015-05-26
2015-05-26 19:23:00  263  268.0  2015-05-26
2015-05-26 19:28:00  263  248.0  2015-05-26
2015-05-26 19:33:00  263  240.0  2015-05-26
2015-05-26 19:38:00  263  236.0  2015-05-26
2015-05-26 1